In [66]:
import pandas as pd
import numpy as np
import os
import warnings 
warnings.filterwarnings("ignore")

from sklearn.metrics import classification_report


os.chdir("F://NMIMS//sem2//ML//DATASETS FOR ML//heart")

In [93]:
#Problem 5: Predict Heart disease (0 to 4)
df = pd.read_csv("heart.csv")

In [94]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [123]:
df["ca"].value_counts()

0    175
1     65
2     38
3     20
4      5
Name: ca, dtype: int64

In [96]:
df.dtypes

age           int64
sex           int64
cp            int64
trestbps      int64
chol          int64
fbs           int64
restecg       int64
thalach       int64
exang         int64
oldpeak     float64
slope         int64
ca            int64
thal          int64
target        int64
dtype: object

In [97]:
df.shape

(303, 14)

In [98]:
df.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [99]:
from sklearn.preprocessing import MinMaxScaler
cols = list(df)
scalar = MinMaxScaler()
df1 = scalar.fit_transform(df)


In [100]:
df2 = pd.DataFrame(df1,columns=cols)
df2.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,0.708333,1.0,1.000000,0.481132,0.244292,1.0,0.0,0.603053,0.0,0.370968,0.0,0.0,0.333333,1.0
1,0.166667,1.0,0.666667,0.339623,0.283105,0.0,0.5,0.885496,0.0,0.564516,0.0,0.0,0.666667,1.0
2,0.250000,0.0,0.333333,0.339623,0.178082,0.0,0.0,0.770992,0.0,0.225806,1.0,0.0,0.666667,1.0
3,0.562500,1.0,0.333333,0.245283,0.251142,0.0,0.5,0.816794,0.0,0.129032,1.0,0.0,0.666667,1.0
4,0.583333,0.0,0.000000,0.245283,0.520548,0.0,0.5,0.702290,1.0,0.096774,1.0,0.0,0.666667,1.0


In [101]:
X = df2.drop(["target"],1)
y = df2["target"]

# OLS stats model (p value) backward elimination


In [103]:

import statsmodels.api as sm
cols = list(X)
p_max = 1
while (len(cols)>0):
   p=[]
   X_1= X[cols]
   X_1 = sm.add_constant(X_1)
   model = sm.OLS(y,X_1).fit()
   p = list(model.pvalues.values[1:])
   p = pd.Series(p,index = cols)
   p_max = max(p)
   features_with_p_max = p.idxmax()
   if (p_max>0.05):
       cols.remove(features_with_p_max)
   else:
       break
selected_features = cols

# Selected Features

In [104]:
selected_features 

['sex', 'cp', 'thalach', 'exang', 'oldpeak', 'ca', 'thal']

In [111]:
df_new = df2[selected_features]

In [112]:
df_new.head()

,sex,cp,thalach,exang,oldpeak,ca,thal
0,1.0,1.000000,0.603053,0.0,0.370968,0.0,0.333333
1,1.0,0.666667,0.885496,0.0,0.564516,0.0,0.666667
2,0.0,0.333333,0.770992,0.0,0.225806,0.0,0.666667
3,1.0,0.333333,0.816794,0.0,0.129032,0.0,0.666667
4,0.0,0.000000,0.702290,1.0,0.096774,0.0,0.666667


In [128]:
X = df_new.drop(["ca"],1)
y = df["ca"]


In [129]:
from sklearn.model_selection import train_test_split as tts
X_train,X_test,y_train,y_test = tts(X,y,random_state =42)

In [130]:
##--------->Logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score,accuracy_score


LogReg = LogisticRegression()
LogReg.fit(X_train,y_train)
y_pred = LogReg.predict(X_test)
accuracy_score(y_pred,y_test)

0.5131578947368421

In [131]:
#-----------> Random Forest
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(X_train,y_train)
y_pred = rfc.predict(X_test)
accuracy_score(y_pred,y_test)

0.5263157894736842

In [144]:
#----------->KNN Classifier
from sklearn.neighbors import KNeighborsClassifier 
knn = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2) 

knn.fit(X_train,y_train)
y_pred = knn.predict(X_test)
accuracy_score(y_pred,y_test)

0.4868421052631579

### KNN Classifier with CV 

In [145]:
#----------->KNN Classifier
from sklearn.model_selection import cross_val_score

cvs = cross_val_score(knn, X,y,cv=10,scoring="accuracy")
cvs.mean()

0.5170566502463053

In [146]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[32,  8,  1,  0,  0],
       [10,  4,  0,  1,  0],
       [ 7,  6,  1,  0,  0],
       [ 1,  2,  1,  0,  0],
       [ 1,  1,  0,  0,  0]], dtype=int64)

In [147]:
print(classification_report(y_test,y_pred ))
#conf(lda,X_test_sfs, y_test)

              precision    recall  f1-score   support

           0       0.63      0.78      0.70        41
           1       0.19      0.27      0.22        15
           2       0.33      0.07      0.12        14
           3       0.00      0.00      0.00         4
           4       0.00      0.00      0.00         2

   micro avg       0.49      0.49      0.49        76
   macro avg       0.23      0.22      0.21        76
weighted avg       0.44      0.49      0.44        76



# Recursive feature elimination


In [167]:
# Feature extraction
from sklearn.feature_selection import RFE

model = LogisticRegression()
rfe = RFE(model, 3)
features = rfe.fit(X, y)
print("Num Features: %s" % (features.n_features_))
print("Selected Features: %s" % (features.support_))
print("Feature Ranking: %s" % (features.ranking_))

Num Features: 3
Selected Features: [False False  True False  True  True]
Feature Ranking: [4 2 1 3 1 1]


In [168]:
cols = list(X)

In [169]:
cols = pd.Series(cols)

# Selected features through RFE

In [170]:
cols[features.support_]

2    thalach
4    oldpeak
5       thal
dtype: object